Convert each cube in the dataset into its mean value. Not particuarly useful but it requires a bunch of numpy calculations + all of the data to be loaded so it's a decent test.

In [1]:
import iris
from distributed import Client

c = Client('172.31.18.5:8786')
c

<Client: scheduler="172.31.18.5:8786" processes=40 cores=40>

In [2]:
cubes = c.get_dataset('remote_mogreps')

In [3]:
cubes.take(20)

(<iris 'Cube' of wet_bulb_freezing_level_altitude / (m) (latitude: 600; longitude: 800)>,
 <iris 'Cube' of wet_bulb_potential_temperature / (K) (time: 2; pressure: 3; latitude: 600; longitude: 800)>,
 <iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 2; latitude: 600; longitude: 800)>,
 <iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
 <iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
 <iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
 <iris 'Cube' of air_temperature / (K) (time: 2; pressure: 16; latitude: 600; longitude: 800)>,
 <iris 'Cube' of dew_point_temperature / (K) (latitude: 600; longitude: 800)>,
 <iris 'Cube' of fog_area_fraction / (1) (latitude: 600; longitude: 800)>,
 <iris 'Cube' of geopotential_height / (m) (time: 2; pressure: 9; latitude: 600; longitude: 800)>,
 <iris 'Cube' of high_type_cloud_area_fraction / (1) (latitude: 600; longitude: 800)>,
 <iris 'Cube' of low_type_cloud_area_fraction / (1) 

This sample analysis requires all of the data to be loaded. If working with s3fs mounted data this process is either borked or it takes so long that I've never had the patience to see it through. If the data is already loaded onto the nodes it's much more reasonable, but it still seems temperamental when shuffling cubes around.

In [4]:
def meaner(cube):
    cube.collapsed(['latitude', 'longitude'], iris.analysis.MEAN)
    for coord in cube.dim_coords:
        cube = cube.collapsed(coord.name(), iris.analysis.MEAN)
    return cube.data

In [5]:
meaned_cubes = cubes.map(meaner)

In [6]:
meaned_cubes.take(10)

(array(1734.5778125),
 array(286.42127591145834),
 array(100690.60130000001),
 array(277.743828125),
 array(278.213203125),
 array(278.60755859375),
 array(240.6882495686849),
 array(274.6958984375),
 array(0.01849791646003723),
 array(5433.2906414351855))